In [205]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
os.makedirs("images", exist_ok=True)



## Génération des images

In [168]:
def generate_image(a, b, c, idx):
    x = np.linspace(-10, 10, 100)
    y = a * x**2 + b * x + c
    
    plt.figure(figsize=(2, 2), dpi=32)
    plt.plot(x, y, color='black', linewidth=3)
    plt.ylim(-200, 200)
    plt.axis('off')
    plt.tight_layout()
    path = f"images/img_{idx:03}.png"
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close()
    return path

In [190]:
from matplotlib.tri import Triangulation

def generate_3dim_image(a, b, c, ax, idx, rotation):
    """Generate 3D plot on given axis instead of creating new figure"""
    x = np.linspace(-6, 6, 30)
    y = np.linspace(-6, 6, 30)
    X, Y = np.meshgrid(x, y)
    Z = a * (X + Y)**2 + b * (X + Y) + c
    
    tri = Triangulation(X.ravel(), Y.ravel())
    
    ax.plot_trisurf(tri, Z.ravel(), cmap='cool', edgecolor='none', alpha=0.8)
    ax.grid(False)
    ax.xaxis.pane.set_visible(False)
    ax.yaxis.pane.set_visible(False)
    ax.zaxis.pane.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    match rotation:
        case 'XY':
            ax.set_title('Surface Plot from XY')
            ax.view_init(elev=90, azim=-90)
        case 'XZ':
            ax.set_title('Surface Plot from XZ')
            ax.view_init(elev=0, azim=0)
        case 'YZ':
            ax.set_title('Surface Plot from YZ')
            ax.view_init(elev=0, azim=-90)

def create_combined_3d_plot(a, b, c, idx):
    """Create combined 3D plot and save it"""
    # Create the subplot figure with 3 3D axes
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={'projection': '3d'})
    
    # Plot the three rotations on the respective axes
    generate_3dim_image(a, b, c, axes[0], idx, 'XY')
    generate_3dim_image(a, b, c, axes[1], idx, 'XZ')
    generate_3dim_image(a, b, c, axes[2], idx, 'YZ')
    
    plt.tight_layout()
    path = f"images3dim/img_{idx:03}.png"
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close()
    return path

## Génération du fichier paramètres

In [28]:
def generate_excel(param_list, path='params.xlsx'):
    df = pd.DataFrame(param_list, columns=['a', 'b', 'c'])
    df.to_excel(path, index=False)


In [ ]:
def generate_all_images_from_excel(excel_path='params.xlsx'):
    df = pd.read_excel(excel_path)
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        generate_image(row['a'], row['b'], row['c'], idx)

def generate_new_images_from_excel(excel_path='params.xlsx'):
    df = pd.read_excel(excel_path)
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        create_combined_3d_plot(row['a'], row['b'], row['c'], idx)

## Construction du MLP

In [207]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [249]:
class ImageRegressionDataset(Dataset):
    def __init__(self, excel_path, image_folder):
        self.df = pd.read_excel(excel_path)
        self.image_folder = image_folder
        self.transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),  
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        params = self.df.iloc[idx][['a', 'b', 'c']].values.astype(np.float32)
        img_path = os.path.join(self.image_folder, f"img_{idx:03}.png")
        image = Image.open(img_path) #.convert('L')  
        image = self.transform(image)
        return torch.tensor(params), image


In [252]:
n_params = 3
output_size = 128

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_params, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_size * output_size * 3), # 3 for rgb
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        batch_size = x.shape[0]
        print(output.reshape(batch_size, output_size, output_size, 3).shape)
        return output.reshape(batch_size, output_size, output_size, 3)


In [237]:
def main_training(training_data='images'):
    dataset = ImageRegressionDataset('params.xlsx',training_data)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    model = MLP()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    patience = 5
    best = 1e5
    for epoch in range(100):
        for inputs, targets in dataloader:
            targets = targets.view(inputs.size(0), -1)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if loss > best:
                patience -= 1
        else:
            patience = 5
            best = loss
        if patience == 0:
            torch.save(model.state_dict(), f'mlp_weights_epoch{epoch+1}.pth')
            break

        print(f"Epoch {epoch+1}, Loss = {loss.item():.4f}")

    torch.save(model.state_dict(), 'mlp_weights.pth')

## Inference

In [ ]:
def main_inference(a, b, c, weights='mlp_weights.pth',output_folder='predictions'):
    os.makedirs(output_folder, exist_ok=True)

    model = MLP()
    model.load_state_dict(torch.load(weights))
    model.eval()

    with torch.no_grad():
        input_tensor = torch.tensor([[a, b, c]], dtype=torch.float32)
        output = model(input_tensor).view(128, 128).numpy()
        output[output < 0.8] *= 0.2 
        plt.imshow(output, cmap='gray') # a modifier
        plt.axis('off')
        image_path = os.path.join(output_folder, f"prediction_a{a}_b{b}_c{c}.png")
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

    print("Image sauvegardée dans le fichier predictions")

## Entraînement du modèle

In [39]:
a_vals = np.linspace(1, 3, 10)
b_vals = np.linspace(10, 30, 10)
c_vals = np.linspace(1, 3, 10)
param_list = [(a, b, c) for a in a_vals for b in b_vals for c in c_vals]
generate_excel(param_list)
generate_all_images_from_excel()
main_training()


100%|██████████| 1000/1000 [00:17<00:00, 56.87it/s]


Epoch 1, Loss = 0.0145
Epoch 2, Loss = 0.0114
Epoch 3, Loss = 0.0105
Epoch 4, Loss = 0.0086
Epoch 5, Loss = 0.0079
Epoch 6, Loss = 0.0068
Epoch 7, Loss = 0.0059
Epoch 8, Loss = 0.0049
Epoch 9, Loss = 0.0054
Epoch 10, Loss = 0.0043
Epoch 11, Loss = 0.0043
Epoch 12, Loss = 0.0029
Epoch 13, Loss = 0.0032
Epoch 14, Loss = 0.0029
Epoch 15, Loss = 0.0027
Epoch 16, Loss = 0.0026
Epoch 17, Loss = 0.0021
Epoch 18, Loss = 0.0022
Epoch 19, Loss = 0.0017
Epoch 20, Loss = 0.0023
Epoch 21, Loss = 0.0017
Epoch 22, Loss = 0.0015
Epoch 23, Loss = 0.0012
Epoch 24, Loss = 0.0018
Epoch 25, Loss = 0.0018
Epoch 26, Loss = 0.0017
Epoch 27, Loss = 0.0011
Epoch 28, Loss = 0.0012
Epoch 29, Loss = 0.0013
Epoch 30, Loss = 0.0009
Epoch 31, Loss = 0.0014
Epoch 32, Loss = 0.0006
Epoch 33, Loss = 0.0014
Epoch 34, Loss = 0.0009
Epoch 35, Loss = 0.0009
Epoch 36, Loss = 0.0014
Epoch 37, Loss = 0.0010
Epoch 38, Loss = 0.0016
Epoch 39, Loss = 0.0007
Epoch 40, Loss = 0.0009
Epoch 41, Loss = 0.0013
Epoch 42, Loss = 0.0007
E

In [194]:
#generate_new_images_from_excel()
main_training(training_data='images3dim')

Epoch 1, Loss = 0.0078
Epoch 2, Loss = 0.0027
Epoch 3, Loss = 0.0007
Epoch 4, Loss = 0.0004
Epoch 5, Loss = 0.0001
Epoch 6, Loss = 0.0002
Epoch 7, Loss = 0.0001
Epoch 8, Loss = 0.0001
Epoch 9, Loss = 0.0001
Epoch 10, Loss = 0.0001
Epoch 11, Loss = 0.0002
Epoch 12, Loss = 0.0001
Epoch 13, Loss = 0.0001
Epoch 14, Loss = 0.0001
Epoch 15, Loss = 0.0001
Epoch 16, Loss = 0.0001
Epoch 17, Loss = 0.0001
Epoch 18, Loss = 0.0001
Epoch 19, Loss = 0.0001
Epoch 20, Loss = 0.0000
Epoch 21, Loss = 0.0000
Epoch 22, Loss = 0.0001
Epoch 23, Loss = 0.0001
Epoch 24, Loss = 0.0000
Epoch 25, Loss = 0.0001
Epoch 26, Loss = 0.0001
Epoch 27, Loss = 0.0000
Epoch 28, Loss = 0.0002
Epoch 29, Loss = 0.0000
Epoch 30, Loss = 0.0001


KeyboardInterrupt: 

In [253]:
main_training(training_data='images3dim')

torch.Size([8, 128, 128, 3])


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 65536])) that is different to the input size (torch.Size([8, 128, 128, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (3) must match the size of tensor b (65536) at non-singleton dimension 3

## Prédictions

In [202]:
main_inference(8,2,3, weights='mlp_weights_epoch20.pth')

Image sauvegardée dans le fichier predictions
